In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1_Загрузка_данных**

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df.head(4)

In [ ]:
df.dtypes

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, models, datasets
import torch.nn as nn
import torch.optim as optim

import os
import time
import glob
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
%matplotlib inline
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# **2_Разбивка_данных_train/val**

In [ ]:
# Разбивка данных на основе индекса
from sklearn.model_selection import train_test_split
trn_ids, val_ids = train_test_split(df.index, test_size=0.3, random_state=99)
# типа разбивка df на train_df и val_df на основе разбивки train_test_split
trn_df, val_df = df[df.index.isin(trn_ids)], df[df.index.isin(val_ids)]
#print(trn_df, val_df)
len(trn_df), len(val_df)

In [ ]:
# trn_ids, val_ids = train_test_split(trn_df.index, test_size=0.3, random_state=99)
# trn_df, val_df = df[df.index.isin(trn_ids)], df[df.index.isin(val_ids)]
# len(trn_df), len(val_df)

In [ ]:
# сбрасываем индексы
trn_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

In [ ]:
trn_df.label.value_counts()

In [ ]:
val_df.label.value_counts()

**определили trn_df, val_df**

# **3_Компановка_даннх**

In [ ]:
t = np.array(df.iloc[np.random.randint(300),1:]).reshape(28,-1)
pic=[]
for i in np.arange(3):
    pic.append(t)
arrImg = np.array(pic).transpose(1, 2, 0)
print(arrImg.shape)

# должен быть массив array, форма (28, 28, 3)
plt.imshow(arrImg)

In [ ]:
# class Asbooka(Dataset):
#     def __init__(self, df):
#         self.df = df
#         self.pics = np.array(df.iloc[:,1:])
#         self.targets = np.array(df.iloc[:,0])
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#     def __len__(self): return len(self.targets)
#     def __getitem__(self, ix):
        
#         f = self.pics[ix].reshape(28,-1)
#         pic=[]
#         for i in np.arange(3):
#             pic.append(f)
#         im = np.array(torch.tensor(pic).permute(1,2,0))
#         im = cv2.resize(im.astype('float32'), (224,224))
#         im = torch.tensor(im/255)
#         im = im.permute(2,0,1)
#         im = self.normalize(im) 
#         target = self.targets[ix]
#         return im.float(), torch.tensor(target)

In [ ]:
# def get_data():
#     train = Asbooka(trn_df)
#     trn_dl = DataLoader(train, batch_size=64, shuffle=True, drop_last = True)
#     val = Asbooka(val_df)
#     val_dl = DataLoader(val, batch_size=32, shuffle=True, drop_last = True)
#     return train, trn_dl, val_dl
# train, trn_dl, val_dl = get_data()

In [ ]:
# # train = Asbooka(val_dl)
# for i in train:
#     print(i)
#     break

# **4_Вытаскиваем_модель,_меняем_слои**

In [ ]:
from torch.optim import SGD, Adam
import torch.nn as nn
def get_model():
    class neuralnet(nn.Module):
        def __init__(self):
            super().__init__()
            model = models.resnet18()
            #------------------------------
            # меняем выходной слой
            model.fc = nn.Linear(in_features=512, out_features=10)
#             model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
#             model.fc = nn.Sequential(
#               nn.Flatten(),
#               nn.Linear(512, 128), # 512 for resnet18 or 2048 for resnet 50
#               nn.ReLU(inplace=True),
#               nn.Dropout(.2),
#               nn.Linear(128, 1),
#               nn.Sigmoid()
#             )
            #------------------------------
            # замораживаем слои
            freeze_layers = [model.layer1, model.layer2, model.conv1, model.bn1]#, model.layer3]
            for layer in freeze_layers:
               for param in layer.parameters():
                 param.requires_grad = False
            #------------------------------
            self.model = model
        def forward(self, x):
            x = self.model(x)
            return x
    model = neuralnet().model

    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.01)
    return model.to(device), loss_fn, optimizer
model, loss_fn, optimizer = get_model()

In [ ]:
optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
# здесь разбиваются скорости корректировок для весов и смещений
# optm = torch.optim.SGD ([
#     {'params': model.weight, 'weight_decay': 0.5},
#     {'params': model.bias, 'weight_decay': 0.0}
# ],lr=1e-3)

In [ ]:
# глядим замороженные/размороженные слои
for name, param in model.named_parameters():
    print(name,param.requires_grad)

In [ ]:
model.fc

# **Пример вставкт L1 и L2 регуляризации**

In [ ]:
#     outputs=model(inputs)
#     loss=loss_fn(outputs,labels)
    
#     #Replaces pow(2.0) with abs() for L1 regularization
     
#     l2_lambda = 0.001
#     l2_norm = sum(p.pow(2.0).sum()
#                   for p in model.parameters())
 
#     loss = loss + l2_lambda * l2_norm

In [ ]:
# Оптимизатор SGD в PyTorch уже имеет параметр weight_decay, который соответствует L2

In [ ]:
# здесь разбиваются скорости корректировок для весов и смещений
# optm = torch.optim.SGD ([
#     {'params': model.weight, 'weight_decay': 0.5},
#     {'params': model.bias, 'weight_decay': 0.0}
# ],lr=1e-3)

# **5_Аугментация**

In [ ]:
IMG_SIZE = 28
# Transformations for the train
train_trans = transforms.Compose(([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
#     transforms.RandomVerticalFlip(0.4), 
#     transforms.RandomHorizontalFlip(0.4),  
#     transforms.RandomRotation(50,expand=True),
#     transforms.RandomCrop(IMG_SIZE), 
    transforms.ToTensor(), # делится на 255
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
#     transforms.Normalize((0.5,), (0.5,))
]))

# Transformations for the validation & test sets
val_trans = transforms.Compose(([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(), # divides by 255
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
   # transforms.Normalize((0.1307,), (0.3081,))
]))

In [ ]:
class Asbooka(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms=transforms
        self.pics = np.array(df.iloc[:,1:])
        self.targets = np.array(df.iloc[:,0])
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    def __len__(self): return len(self.df)
    def __getitem__(self, ix):
        picture = self.pics[ix]
#         pic=[]
#         for i in np.arange(3):
#           pic.append(picture)
#         print(pic)
        img = np.array(picture).astype(np.uint8)
        img = np.repeat(img.reshape(28,28,1), 3, axis=2) # делаем форму (28,28,3) из (28,28,1)
#         print(img.shape)
#         print(np.repeat(img.reshape(1,28,28), 3, axis=0).shape)
        if self.transforms:
          img=self.transforms(img).to(device)
#         print(img.shape)
        target = self.targets[ix]
        return img, torch.tensor(target).to(device)
    

In [ ]:
def get_data():
    train = Asbooka(trn_df, train_trans)
    trn_dl = DataLoader(train, batch_size=254, shuffle=True, drop_last = True)
    val = Asbooka(val_df, val_trans)
    val_dl = DataLoader(val, batch_size=64, shuffle=False, drop_last = True)
    return train, trn_dl, val_dl
train, trn_dl, val_dl = get_data()

In [ ]:
def show_img(img, title):
#   plt.figure(figsize=(20,20))
    npimg=img.cpu().numpy().transpose(1,2,0)
    plt.title(title)
    plt.imshow(npimg)#.transpose(1,2,0))
    plt.show()
    
kl = iter(train)

In [ ]:
rr = next(kl)
print(rr[0].shape)
show_img(rr[0], rr[1])

# **5_Функции_для_train_val_вычисляющие_loss_&_accuracy**

In [ ]:
def train_batch_loss_acc(x, y, model, opt, loss_fn):
    model.train()
    prediction = model(x)
    
#     print(prediction.max(1)[1])
#     print(y)
    #-------------------------
    #loss
    loss = loss_fn(prediction, y)
#     print(loss)
#     loss = CrossEntroptLoss(prediction,y)
#     print( CrossEntroptLoss(prediction,y))
    #-------------------------
    #добавление L1-регуляризации
#     l2_lambda = 0.01
#     l2_norm = sum(p.abs().sum() for p in model.parameters())
#     batch_loss = loss + l2_lambda * l2_norm
    #-------------------------
    #добавление L2-регуляризации
    l2_lambda = 0.0001
    l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
    batch_loss = loss + l2_lambda * l2_norm
    #-------------------------
    #accuracy
    max_values, argmaxes = prediction.max(-1)
    is_correct = np.mean((argmaxes == y).double().cpu().numpy())
    #_________________________
#     print(batch_loss)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item(), is_correct

In [ ]:
def val_batch_loss_acc(x, y, model, opt, loss_fn):
    model.eval()
    with torch.no_grad():
        prediction = model(x)
    #-------------------------
    #loss
    batch_loss = loss_fn(prediction, y)
#     batch_loss = CrossEntroptLoss(prediction,y)
    #-------------------------
    #accuracy
    max_values, argmaxes = prediction.max(-1)
    is_correct = np.mean((argmaxes == y).double().cpu().numpy())
    #_________________________
    return batch_loss.item(), is_correct

In [ ]:
# здесь по-моему первого достаточно, добавление второго качество почти не улучшает
from torch.optim.lr_scheduler import StepLR
# scheduler_1 = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.9)
scheduler_2 = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[4], gamma=0.1)
# scheduler_3 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.1, patience = 3)

# **5_Функция потерь CrossEntroptLoss**

In [ ]:
# def CrossEntroptLoss (y_pred, y_true):
#     y_pred = y_pred.max(1)[1]
#     out = (y_pred == y_true).double().numpy()
#     return 1 - np.mean(out)

# **6_Процесс_обучения**

In [ ]:
# scheduler_2.get_last_lr()[0]

In [ ]:
train_losses, train_accuracies = [], []
val_losses, val_accuracies = [], []
for epoch in range(12):
    print(epoch)
    train_epoch_losses, train_epoch_accuracies = [], []
    for ix, batch in enumerate(tqdm(trn_dl)):
        x, y = batch
        batch_loss_tr, batch_accuracy_tr = train_batch_loss_acc(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss_tr)
        train_epoch_accuracies.append(batch_accuracy_tr)
        if ix%10 == 0:
            print(f'{ix} batch_train_loss: {np.array(batch_loss_tr)}')
            print(f'{ix} sum_train_loss: {np.array(train_epoch_losses).mean()}')
            print(f'{ix} batch_train_acc: {np.array(batch_accuracy_tr)}')
            print(f'{ix} sum_train_acc: {np.array(train_epoch_accuracies).mean()}', end='\r')
    train_losses.append(np.array(train_epoch_losses).mean())
    train_accuracies.append(np.array(train_epoch_accuracies).mean())
    
    val_epoch_losses, val_epoch_accuracies = [], []
    for ix, batch in enumerate(tqdm(val_dl)):
        x, y = batch
        batch_loss_val, batch_accuracy_val = val_batch_loss_acc(x, y, model, optimizer, loss_fn)
        val_epoch_losses.append(batch_loss_val)
        val_epoch_accuracies.append(batch_accuracy_val)
        if ix%10 == 0:
            print(f'{ix} batch_val_loss: {np.array(batch_loss_val)}')
            print(f'{ix} sum_val_loss: {np.array(val_epoch_losses).mean()}')
            print(f'{ix} batch_val_acc: {np.array(batch_accuracy_val)}')
            print(f'{ix} sum_val_acc: {np.array(val_epoch_accuracies).mean()}', end='\r')
    #---------------------------------
    # изменение шага
    scheduler_2.step() # меняем_шаг
    print(scheduler_2.get_last_lr()[0]) # выводим_значение_шага
    #---------------------------------    
    val_losses.append(np.array(val_epoch_losses).mean())
    val_accuracies.append(np.array(val_epoch_accuracies).mean())

    
    print(f'train_losses: {train_losses}, train_accuracies: {train_accuracies}')
    print(f'val_losses: {val_losses}, val_accuracies: {val_accuracies}')

# **7_Вывод_результатов**

In [ ]:
x = np.arange(0,12)
y1 = train_losses
y2 = val_losses
plt.figure(figsize=(12, 7))
plt.plot(x, y1, 'o-r', alpha=0.7, label="train_losses", lw=5, mec='b', mew=2, ms=10)
plt.plot(x, y2, 'v-.g', label="val_losses", mec='r', lw=2, mew=2, ms=12)
plt.legend()
plt.grid(True)

In [ ]:
x = np.arange(0,12)
y1 = train_accuracies
y2 = val_accuracies
plt.figure(figsize=(12, 7))
plt.plot(x, y1, 'o-r', alpha=0.7, label="train_acc", lw=5, mec='b', mew=2, ms=10)
plt.plot(x, y2, 'v-.g', label="val_acc", mec='r', lw=2, mew=2, ms=12)
plt.legend()
plt.grid(True)

# **8_Предсказание**

In [ ]:
# random выбор картинок
df_test = df.sample(6)
df_test

In [ ]:
# преобразования для подачи в модель
t = Asbooka(df_test, val_trans)
t_dl = DataLoader(t, batch_size=6, shuffle=True, drop_last = True)

# разделяем картинки и true
for i in t_dl:
    x,y = i
#     print(x)
    print(y)
    
# блок предсказания класса
@torch.no_grad()
def pred(x, model):
    model.eval()
    with torch.no_grad():
        prediction = model(x)
    pred_list = prediction.max(1)[1].detach().cpu().numpy()
    return x.cpu().numpy(), pred_list

# делаем список картинка + предсказание
images, predictions = pred(x, model)
data = list(zip(images, predictions))

fig, ax = plt.subplots(2, 3, figsize=(10, 7))
axs = [ax[i, j] for i in range(2) for j in range(3)]
for _,i in enumerate(data):
    axs[_].set_title(i[1], fontweight ="bold")
    axs[_].imshow(i[0].transpose(1, 2, 0))
#     arrImg = i[0].transpose(1, 2, 0)
#     plt.title(i[1], fontweight ="bold") 
#     plt.imshow(arrImg)

# **Шаг 9: Передаем данные**

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
test.shape

In [ ]:
test.head(5)

In [ ]:
class Asbookatest(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms=transforms
        self.pics = np.array(df)
#         self.targets = np.array(df.iloc[:,0])
#         self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    def __len__(self): return len(self.df)
    def __getitem__(self, ix):
        picture = self.pics[ix]
#         pic=[]
#         for i in np.arange(3):
#           pic.append(picture)
#         print(pic)
        img = np.array(picture).astype(np.uint8)
        img = np.repeat(img.reshape(28,28,1), 3, axis=2) # делаем форму (28,28,3) из (28,28,1)
#         print(img.shape)
#         print(np.repeat(img.reshape(1,28,28), 3, axis=0).shape)
        if self.transforms:
          img=self.transforms(img).to(device)
#         print(img.shape)
#         target = self.targets[ix]
        return img#, torch.tensor(target)
    

In [ ]:
test_test = Asbookatest(test, val_trans)
test_loader = DataLoader(test_test, batch_size=1000, shuffle=False)

In [ ]:
predictions = []

with torch.no_grad():
    model.eval()  # Set model to evaluation mode
    for data in tqdm(test_loader):
        inputs = data#[0]
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.tolist())

In [ ]:
predictions

In [ ]:
submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
print(submission)

In [ ]:
pred = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), 'Label': predictions})
submission = pd.merge(submission['ImageId'], pred, on='ImageId')
submission.to_csv('submission.csv', index=False)

In [ ]:
pred